In [1]:
import yfinance as yf

In [2]:
#constraints

budget = 20000
risk_tolerance = 0.5

In [3]:
#calculating the return
def daily_returns(yesterday, today):
    if yesterday == 0:
        return 0
    return (today - yesterday) / yesterday

def expected_return(days, list_of_returns):
    return sum(list_of_returns) / days

def variance_of_asset(list_of_returns, days):
    mean_return = expected_return(days, list_of_returns)
    return sum((r - mean_return) ** 2 for r in list_of_returns) / days

def covariance(asset1_returns, asset2_returns, days):
    mean_asset1 = expected_return(days, asset1_returns)
    mean_asset2 = expected_return(days, asset2_returns)
    return sum((asset1_returns[i] - mean_asset1) * (asset2_returns[i] - mean_asset2) for i in range(days)) / days

In [4]:
import pandas as pd

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX', 'AMD', 'IBM']
data = yf.download(tickers, start="2023-01-01", end="2025-01-01")

close_prices = data['Close']

#close_prices = close_prices.pct_change().dropna()

list_of_returns_aaple = []
for i in range(1, len(close_prices['AAPL'])):
    daily_return = daily_returns(close_prices['AAPL'].iloc[i-1], close_prices['AAPL'].iloc[i])
    list_of_returns_aaple.append(daily_return)
average_return_aaple = expected_return(len(list_of_returns_aaple), list_of_returns_aaple)
print(f"Average daily return for AAPL: {average_return_aaple:.4f}")
variance_of_aaple = variance_of_asset(list_of_returns_aaple, len(list_of_returns_aaple))
print(f"Variance of daily returns for AAPL: {variance_of_aaple:.4f}")


list_of_returns_nvda = []
for i in range(1, len(close_prices['NVDA'])):
    daily_return = daily_returns(close_prices['NVDA'].iloc[i-1], close_prices['NVDA'].iloc[i])
    list_of_returns_nvda.append(daily_return)
average_return_nvda = expected_return(len(list_of_returns_nvda), list_of_returns_nvda)
print(f"Average daily return for NVDA: {average_return_nvda:.4f}")
variance_of_nvda = variance_of_asset(list_of_returns_nvda, len(list_of_returns_nvda))
print(f"Variance of daily returns for NVDA: {variance_of_nvda:.4f}")

list_of_returns_msft = []
for i in range(1, len(close_prices['MSFT'])):
    daily_return = daily_returns(close_prices['MSFT'].iloc[i-1], close_prices['MSFT'].iloc[i])
    list_of_returns_msft.append(daily_return)
average_return_msft = expected_return(len(list_of_returns_msft), list_of_returns_msft)
print(f"Average daily return for MSFT: {average_return_msft:.4f}")
variance_of_msft = variance_of_asset(list_of_returns_msft, len(list_of_returns_msft))
print(f"Variance of daily returns for MSFT: {variance_of_msft:.4f}")

list_of_returns_amzn = []
for i in range(1, len(close_prices['AMZN'])):
    daily_return = daily_returns(close_prices['AMZN'].iloc[i-1], close_prices['AMZN'].iloc[i])
    list_of_returns_amzn.append(daily_return)
average_return_amzn = expected_return(len(list_of_returns_amzn), list_of_returns_amzn)
print(f"Average daily return for AMZN: {average_return_amzn:.4f}")
variance_of_amzn = variance_of_asset(list_of_returns_amzn, len(list_of_returns_amzn))
print(f"Variance of daily returns for AMZN: {variance_of_amzn:.4f}")

list_of_returns_googl = []
for i in range(1, len(close_prices['GOOGL'])):
    daily_return = daily_returns(close_prices['GOOGL'].iloc[i-1], close_prices['GOOGL'].iloc[i])
    list_of_returns_googl.append(daily_return)
average_return_googl = expected_return(len(list_of_returns_googl), list_of_returns_googl)
print(f"Average daily return for GOOGL: {average_return_googl:.4f}")
variance_of_googl = variance_of_asset(list_of_returns_googl, len(list_of_returns_googl))
print(f"Variance of daily returns for GOOGL: {variance_of_googl:.4f}")

list_of_returns_tsla = []
for i in range(1, len(close_prices['TSLA'])):
    daily_return = daily_returns(close_prices['TSLA'].iloc[i-1], close_prices['TSLA'].iloc[i])
    list_of_returns_tsla.append(daily_return)
average_return_tsla = expected_return(len(list_of_returns_tsla), list_of_returns_tsla)
print(f"Average daily return for TSLA: {average_return_tsla:.4f}")
variance_of_tsla = variance_of_asset(list_of_returns_tsla, len(list_of_returns_tsla))
print(f"Variance of daily returns for TSLA: {variance_of_tsla:.4f}")

list_of_returns_meta = []
for i in range(1, len(close_prices['META'])):
    daily_return = daily_returns(close_prices['META'].iloc[i-1], close_prices['META'].iloc[i])
    list_of_returns_meta.append(daily_return)
average_return_meta = expected_return(len(list_of_returns_meta), list_of_returns_meta)
print(f"Average daily return for META: {average_return_meta:.4f}")
variance_of_meta = variance_of_asset(list_of_returns_meta, len(list_of_returns_meta))
print(f"Variance of daily returns for META: {variance_of_meta:.4f}")

list_of_returns_nflx = []
for i in range(1, len(close_prices['NFLX'])):
    daily_return = daily_returns(close_prices['NFLX'].iloc[i-1], close_prices['NFLX'].iloc[i])
    list_of_returns_nflx.append(daily_return)
average_return_nflx = expected_return(len(list_of_returns_nflx), list_of_returns_nflx)
print(f"Average daily return for NFLX: {average_return_nflx:.4f}")
variance_of_nflx = variance_of_asset(list_of_returns_nflx, len(list_of_returns_nflx))
print(f"Variance of daily returns for NFLX: {variance_of_nflx:.4f}")

list_of_returns_ibm = []
for i in range(1, len(close_prices['IBM'])):
    daily_return = daily_returns(close_prices['IBM'].iloc[i-1], close_prices['IBM'].iloc[i])
    list_of_returns_ibm.append(daily_return)
average_return_ibm = expected_return(len(list_of_returns_ibm), list_of_returns_ibm)
print(f"Average daily return for IBM: {average_return_ibm:.4f}")
variance_of_ibm = variance_of_asset(list_of_returns_ibm, len(list_of_returns_ibm))
print(f"Variance of daily returns for IBM: {variance_of_ibm:.4f}")

list_of_returns_amd = []
for i in range(1, len(close_prices['AMD'])):
    daily_return = daily_returns(close_prices['AMD'].iloc[i-1], close_prices['AMD'].iloc[i])
    list_of_returns_amd.append(daily_return)
average_return_amd = expected_return(len(list_of_returns_amd), list_of_returns_amd)
print(f"Average daily return for AMD: {average_return_amd:.4f}")
variance_of_amd = variance_of_asset(list_of_returns_amd, len(list_of_returns_amd))
print(f"Variance of daily returns for AMD: {variance_of_amd:.4f}")




YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed

Average daily return for AAPL: 0.0015
Variance of daily returns for AAPL: 0.0002
Average daily return for NVDA: 0.0050
Variance of daily returns for NVDA: 0.0010
Average daily return for MSFT: 0.0013
Variance of daily returns for MSFT: 0.0002
Average daily return for AMZN: 0.0021
Variance of daily returns for AMZN: 0.0004
Average daily return for GOOGL: 0.0017
Variance of daily returns for GOOGL: 0.0003
Average daily return for TSLA: 0.0033
Variance of daily returns for TSLA: 0.0013
Average daily return for META: 0.0034
Variance of daily returns for META: 0.0006
Average daily return for NFLX: 0.0024
Variance of daily returns for NFLX: 0.0005
Average daily return for IBM: 0.0011
Variance of daily returns for IBM: 0.0002
Average daily return for AMD: 0.0017
Variance of daily returns for AMD: 0.0009


another way of calculating returns

In [5]:
returns = close_prices.pct_change().dropna()
mean_returns = returns.mean()
for ticker in tickers:
    print(f"Average daily return for {ticker}: {mean_returns[ticker]:.4f}")
cov_matrix = returns.cov()
print("\nCovariance Matrix:")   
print(cov_matrix)

Average daily return for AAPL: 0.0015
Average daily return for MSFT: 0.0013
Average daily return for GOOGL: 0.0017
Average daily return for AMZN: 0.0021
Average daily return for TSLA: 0.0033
Average daily return for META: 0.0034
Average daily return for NVDA: 0.0050
Average daily return for NFLX: 0.0024
Average daily return for AMD: 0.0017
Average daily return for IBM: 0.0011

Covariance Matrix:
Ticker      AAPL       AMD      AMZN     GOOGL       IBM      META      MSFT  \
Ticker                                                                         
AAPL    0.000181  0.000136  0.000101  0.000109  0.000025  0.000123  0.000097   
AMD     0.000136  0.000895  0.000245  0.000213  0.000081  0.000295  0.000199   
AMZN    0.000101  0.000245  0.000372  0.000204  0.000042  0.000269  0.000171   
GOOGL   0.000109  0.000213  0.000204  0.000339  0.000021  0.000225  0.000141   
IBM     0.000025  0.000081  0.000042  0.000021  0.000158  0.000053  0.000033   
META    0.000123  0.000295  0.000269  0.0

In [6]:
Sigma = cov_matrix.to_numpy()

QUBO (quadratic unconstrained binary optimization)

In [7]:
import numpy as np
last_day = close_prices.iloc[-1]
asset_prices = [(ticker, last_day[ticker]) for ticker in tickers]
print("\nAsset Prices on Last Day:")
for ticker, price in asset_prices:
    print(f"{ticker}: ${price:.2f}")

#calculate n_i max for each asset:
n_i_max_list = []
for ticker, price in asset_prices:
    n_i_max = budget // price
    n_i_max_list.append((ticker, n_i_max))
    print(f"Maximum number of shares for {ticker} with budget ${budget}: {n_i_max}")



Asset Prices on Last Day:
AAPL: $249.82
MSFT: $419.89
GOOGL: $189.08
AMZN: $219.39
TSLA: $403.84
META: $584.99
NVDA: $134.28
NFLX: $891.32
AMD: $120.79
IBM: $216.93
Maximum number of shares for AAPL with budget $20000: 80.0
Maximum number of shares for MSFT with budget $20000: 47.0
Maximum number of shares for GOOGL with budget $20000: 105.0
Maximum number of shares for AMZN with budget $20000: 91.0
Maximum number of shares for TSLA with budget $20000: 49.0
Maximum number of shares for META with budget $20000: 34.0
Maximum number of shares for NVDA with budget $20000: 148.0
Maximum number of shares for NFLX with budget $20000: 22.0
Maximum number of shares for AMD with budget $20000: 165.0
Maximum number of shares for IBM with budget $20000: 92.0


In [8]:
d_i_list = []
for ticker, n_i_max in n_i_max_list:
    if n_i_max > 0:
        d_i = int(np.floor(np.log2(n_i_max)))
    else:
        d_i = 0
    d_i_list.append((ticker, d_i))
    print(f"Value of d_i for {ticker}: {d_i}")


Value of d_i for AAPL: 6
Value of d_i for MSFT: 5
Value of d_i for GOOGL: 6
Value of d_i for AMZN: 6
Value of d_i for TSLA: 5
Value of d_i for META: 5
Value of d_i for NVDA: 7
Value of d_i for NFLX: 4
Value of d_i for AMD: 7
Value of d_i for IBM: 6


In [9]:
d_values = [d_i for _, d_i in d_i_list]
num_assets = len(d_values)
total_bits = sum(d+ 1 for d in d_values)
print(f"\nTotal number of bits required for all assets: {total_bits}")


Total number of bits required for all assets: 67


constructing the encoding matrix $C$

In [10]:
C = np.zeros((num_assets, total_bits))

col = 0 

for i, d in enumerate(d_values):
    for j in range(d+1):
        C[i, col] = 2 ** j
        col += 1


Calculating 
$$
P'= P/B \\
\mu' = P' \circ \mu \\
\Sigma' = (P' \circ \Sigma)^T \circ P'\\
\mu^{\prime\prime} = C^T \mu' \\
\Sigma'' = C^T \Sigma'C \\
P'' = C^T P'
$$

In [11]:
# List P of current prices of assets 
P = np.array([price for _, price in asset_prices])
P_prime = P / budget
P_double_prime = C.T @ P_prime


mu = np.array([mean_returns[ticker] for ticker in tickers])
mu_prime = P_prime * mu 

mu_prime_series = pd.Series(mu_prime, index=tickers)


print("\nScaled expected return vector µ′:")
print(mu_prime_series)




Scaled expected return vector µ′:
AAPL     0.000019
MSFT     0.000027
GOOGL    0.000016
AMZN     0.000023
TSLA     0.000067
META     0.000099
NVDA     0.000033
NFLX     0.000108
AMD      0.000010
IBM      0.000012
dtype: float64


In [13]:
P_prime_sigma = P_prime * Sigma
Sigma_prime = P_prime_sigma.T * P_prime 

mu_double_prime = C.T @ mu_prime

Sigma_double_prime = C.T @ Sigma_prime @ C
